In [8]:
from daisy import *
import matplotlib.pyplot as plt
import numpy as np

In [15]:
file_path = '/n/groups/htem/ESRF_id16a/tomo_ML/ResolutionEnhancement/jlr54_tests/volumes/'
file_name = 'CBxs_lobV_bottomp100um_30nm_rec_db9_.n5'
in_name = 'volumes/raw'
mask_name = 'volumes/volume_mask'

In [21]:
data = open_ds(file_path+file_name, in_name)
mask = open_ds(file_path+file_name, mask_name)
data.shape

(3216, 3216, 2048)

In [19]:
center_z = data.data_roi.get_center()[2]
slice_size = list(data.data_roi.get_shape())
slice_size[2] = 1
slice_roi = Roi((0, 0, center_z), slice_size)
slice_roi

[0:96480, 0:96480, 30720:30721] (96480, 96480, 1)

In [ ]:
plt.imshow(data[slice_roi])

In [ ]:
plt.imshow(mask[slice_roi])